In [1]:
import pandas as pd
import numpy as np
import matplotlib as pl
import seaborn as sbn
import sqlite3
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
%load_ext sql
%sql sqlite:///database.sqlite

u'Connected: None@database.sqlite'

In [3]:
try:    
    db = sqlite3.connect('database.sqlite')
    c = db.cursor()
    print "Successfully connected to the database"
except IntegrityError:
    print"Did not successfully connect to database"

Successfully connected to the database


In [4]:
#Primary Results:
%sql SELECT * FROM primary_results
%sql ALTER TABLE primary_results ADD COLUMN county_state TEXT;
%sql UPDATE primary_results SET county_state = (county || "_" || state);

#Republican Table:
%sql CREATE TABLE Republicans AS SELECT * FROM primary_results;
%sql DELETE FROM Republicans WHERE(party ='Democrat') 

#Democrat Table:
%sql CREATE TABLE Democrats AS SELECT * FROM primary_results;
%sql DELETE FROM Democrats WHERE(party ='Republican') 

#County Facts:
%sql UPDATE county_facts SET state_abbreviation = NULL WHERE state_abbreviation = '';
%sql DELETE FROM county_facts WHERE state_abbreviation IS NULL;
%sql ALTER TABLE county_facts ADD COLUMN county_state TEXT;
%sql UPDATE county_facts SET county_state = (area_name || "_" || state_abbreviation);

#Output County_Facts_Description as CSV file (.mode csv)
#%sql .output county_facts_dict.csv # command in terminal

Done.
Done.
13212 rows affected.
Done.
4060 rows affected.
Done.
9152 rows affected.
52 rows affected.
52 rows affected.
Done.
3143 rows affected.


[]

In [5]:
# Generate Dataframe
df_primary_results = pd.read_sql_query('SELECT * FROM primary_results', db) #Primary Results 
df_county_facts = pd.read_sql_query('SELECT * FROM county_facts',db) #County Facts
df_republicans = pd.read_sql_query('SELECT * FROM Republicans',db) # Republican Table
df_democrats = pd.read_sql_query('SELECT * FROM Democrats', db) # Democrat Table

#County Facts Dictionary:
df_county_facts_dictionary = pd.read_sql_query('SELECT * FROM county_facts_dictionary ',db)

In [6]:
#shape of tables:
prim = df_primary_results.shape[0]
rep = df_republicans.shape[0]
dem = df_democrats.shape[0]
print "Primary Results..."
print "Total Number of Elements = {}; Republican: {}, Democrat :{}".format(prim,rep,dem)
print "Number of Features =", df_primary_results.shape[1]

#County Facts (shape):
print "\nCounty Facts..."
print "Total Number of Elements =", df_county_facts.shape[0]
print "Number of Features =", df_county_facts.shape[1] -4


Primary Results...
Total Number of Elements = 13212; Republican: 9152, Democrat :4060
Number of Features = 9

County Facts...
Total Number of Elements = 3143
Number of Features = 51


In [7]:
# Primary results
df_primary_results.head()

,state,state_abbreviation,county,fips,party,candidate,votes,fraction_votes,county_state
0,Alabama,AL,Autauga,1001,Republican,Donald Trump,5387,0.445,Autauga_Alabama
1,Alabama,AL,Autauga,1001,Republican,Ted Cruz,2482,0.205,Autauga_Alabama
2,Alabama,AL,Autauga,1001,Republican,Marco Rubio,1785,0.148,Autauga_Alabama
3,Alabama,AL,Autauga,1001,Republican,Ben Carson,1764,0.146,Autauga_Alabama
4,Alabama,AL,Autauga,1001,Republican,John Kasich,421,0.035,Autauga_Alabama


In [8]:
# County Facts data
df_county_facts.head()

,fips,area_name,state_abbreviation,PST045214,PST040210,PST120214,POP010210,AGE135214,AGE295214,AGE775214,...,SBO015207,MAN450207,WTN220207,RTN130207,RTN131207,AFN120207,BPS030214,LND110210,POP060210,county_state
0,1001,Autauga County,AL,55395,54571,1.5,54571,6.0,25.2,13.8,...,31.7,0,0,598175,12003,88157,131,594.44,91.8,Autauga County_AL
1,1003,Baldwin County,AL,200111,182265,9.8,182265,5.6,22.2,18.7,...,27.3,1410273,0,2966489,17166,436955,1384,1589.78,114.6,Baldwin County_AL
2,1005,Barbour County,AL,26887,27457,-2.1,27457,5.7,21.2,16.5,...,27.0,0,0,188337,6334,0,8,884.88,31.0,Barbour County_AL
3,1007,Bibb County,AL,22506,22919,-1.8,22915,5.3,21.0,14.8,...,0.0,0,0,124707,5804,10757,19,622.58,36.8,Bibb County_AL
4,1009,Blount County,AL,57719,57322,0.7,57322,6.1,23.6,17.0,...,23.2,341544,0,319700,5622,20941,3,644.78,88.9,Blount County_AL


In [ ]:
#Identify the Republican winner in each county:
#%sql SELECT state, state_abbreviation, county, fips, party, candidate, MAX(votes), fraction_votes, county_state FROM Republicans GROUP BY county ORDER BY state;
#%sql CREATE TABLE Republican_Winner(state TEXT NOT NULL, state_abbreviation TEXT NOT NULL, county TEXT NOT NULL, fips INTEGER NOT NULL, party TEXT NOT NULL, candiate TEXT NOT NULL, votes INTEGER NOT NULL, fraction_votes INTEGER NOT NULL, county_state TEXT NOT NULL);

#Identify the Democrat winner in each county:
#%sql SELECT state, state_abbreviation, county, fips, party, candidate, MAX(votes), fraction_votes, county_state FROM Democrats GROUP BY county ORDER BY state;
#%sql CREATE TABLE Democrat_Winner(state TEXT NOT NULL, state_abbreviation TEXT NOT NULL, county TEXT NOT NULL, fips INTEGER NOT NULL, party TEXT NOT NULL, candiate TEXT NOT NULL, votes INTEGER NOT NULL, fraction_votes INTEGER NOT NULL, county_state TEXT NOT NULL);


In [ ]:
# Merger Tables
